In [ ]:
import numpy as np
import pandas as pd

def sigmoid(x,w):
  arg = np.dot(x,w.T)
  return 1/(1 + np.exp(-arg))

def pred(y):
  return (np.argmax(y, axis=1) + 1)

def holdout(data, trainp, validp):
  m = np.shape(data)[0]

  train = data[0 : int(np.floor(m*trainp/100))]
  valid = data[int(np.floor(m*trainp/100)) : int(np.floor(m*trainp/100))+int(np.floor(m*validp/100))]
  test = data[int(np.floor(m*trainp/100))+int(np.floor(m*validp/100)) : None]

  y_train = train[:,-1] #shape = (rows,)
  x_train = np.delete(train, -1, axis=1)

  y_valid = valid[:,-1] #shape = (rows,)
  x_valid = np.delete(valid, -1, axis=1)

  y_test = test[:,-1] #shape = (rows,)
  x_test = np.delete(test, -1, axis=1)

  return x_train,x_valid,x_test,y_train,y_valid,y_test

def appendones(x):
  m = np.shape(x)[0]
  return np.concatenate((np.ones((m,1)),x), axis=1)

def one_hot_encode(y):
  len = np.size(y)
  encoded_y = np.zeros((len,3))
  for i in range(len):
    if y[i] == 1:
      encoded_y[i,0] = 1
    if y[i] == 2:
      encoded_y[i,1] = 1
    if y[i] == 3:
      encoded_y[i,2] = 1
  return encoded_y

def performance(y, pred):
  m = np.zeros((3,3)) # confusion matrix
  for p in range(len(pred)):
    if pred[p]==1 and y[p]==1:
      m[0,0]+=1
    if pred[p]==2 and y[p]==2:
      m[1,1]+=1
    if pred[p]==3 and y[p]==3:
      m[2,2]+=1
    if pred[p]==1 and y[p]==2:
      m[1,0]+=1
    if pred[p]==1 and y[p]==3:
      m[2,0]+=1
    if pred[p]==2 and y[p]==1:
      m[0,1]+=1
    if pred[p]==2 and y[p]==3:
      m[2,1]+=1
    if pred[p]==3 and y[p]==1:
      m[0,2]+=1
    if pred[p]==3 and y[p]==2:
      m[1,2]+=1
  ind_accuracy = [m[0,0]/np.sum(m[0,:]), m[1,1]/np.sum(m[1,:]), m[2,2]/np.sum(m[2,:])]
  accuracy = (m[0,0]+m[1,1]+m[2,2])/np.sum(m)
  return ind_accuracy, accuracy

def auto_encoder(x,alpha,lamda,iters):
  W1 = np.random.rand(x.shape[1]-1,x.shape[1])
  W2 = np.random.rand(x.shape[1],x.shape[1]-1)
  
  for i in range(iters):
    H1 = sigmoid(x,W1)
    H2 = sigmoid(H1,W2)
    error  = (H2-x)
    delta2 = error*H2*(1-H2)
    delta1 = np.dot(delta2,W2)*H1*(1-H1)
    W2 -= alpha*(np.dot(delta2.T,H1) + lamda*W2)
    W1 -= alpha*(np.dot(delta1.T,x) + lamda*W1)
    
  return W1,sigmoid(x,W1)

def stacked_auto_encoder(alpha,lamda,iters):
  # pretraining 
  W1,H1 = auto_encoder(x_train,0.06,0.00001,10000)
  W2,H2 = auto_encoder(H1,0.06,0.00001,10000)
  W3,H3 = auto_encoder(H2,0.03,0.00001,10000)

  # fine tuning 
  W4 = np.random.rand(H3.shape[1]-2,H3.shape[1])
  alpha1 = 100*alpha
  for i in range(iters):
    H4 = sigmoid(H3,W4)
    error  = (H4-y_train_coded)
    delta4 = error*H4*(1-H4)
    delta3 = np.dot(delta4,W4)*H3*(1-H3)
    delta2 = np.dot(delta3,W3)*H2*(1-H2)
    delta1 = np.dot(delta2,W2)*H1*(1-H1)
    W4 -= alpha1*(np.dot(delta4.T,H3) + lamda*W4)
    W3 -= alpha*(np.dot(delta3.T,H2) + lamda*W3)
    W2 -= alpha*(np.dot(delta2.T,H1) + lamda*W2)
    W1 -= alpha*(np.dot(delta1.T,x_train) + lamda*W1)
    
  y = sigmoid(sigmoid(sigmoid(sigmoid(x_test,W1),W2),W3),W4)
  return y

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/NNFL Assignments (Aug 2021)/Assignment 2/data5.xlsx')
cols = np.array(data.columns, ndmin=2)
data = data.to_numpy()
data = np.concatenate((cols,data), axis=0)

# shuffle & train-test-valid-split
np.random.seed(0)
np.random.shuffle(data)
x_train, x_valid, x_test, y_train, y_valid, y_test = holdout(data, 70, 10)

# normalizing input data
mu = np.mean(x_train, axis=0)
std = np.std(x_train, axis=0)

x_train = (x_train-mu)/std
x_valid = (x_valid-mu)/std
x_test = (x_test-mu)/std

# one-hot-encoding output data
y_train_coded = one_hot_encode(y_train)
y_valid_coded = one_hot_encode(y_valid)
y_test_coded = one_hot_encode(y_test)

# appending ones (input for bias)
x_train = appendones(x_train)
x_valid = appendones(x_valid)
x_test = appendones(x_test)

m_train = np.size(y_train)
m_valid = np.size(y_valid)
m_test = np.size(y_test)

In [ ]:
y = stacked_auto_encoder(0.0001,0.01,1000)

# performance measures
ind_accuracy, accuracy = performance(y_test, pred(y))
print("accuracy of class 1 = {}".format(ind_accuracy[0]))
print("accuracy of class 2 = {}".format(ind_accuracy[1]))
print("accuracy of class 3 = {}".format(ind_accuracy[2]))
print("overall accuracy of classifier = {}".format(accuracy))

accuracy of class 1 = 0.9
accuracy of class 2 = 1.0
accuracy of class 3 = 0.8947368421052632
overall accuracy of classifier = 0.9285714285714286
